# Green Revenue CFS File Workflow

### Importing libararies

In [1]:
import os
import sys
from dotenv import dotenv_values
config = dotenv_values(".env")

### Set RDP credentials and Initial Parameters

In [2]:
username = config['RDP_USERNAME'] #or replacing with your RDP Username/Machine-ID
password = config['RDP_PASSWORD'] #or replacing with your RDP Password
clientId = config['RDP_APP_KEY'] #or replacing with your RDP App key

RDP_HOST= 'https://api.refinitiv.com'
acccess_token = None
refresh_token = None
expires_in = 0

## <a id="rdp_workflow"></a>RDP APIs Application Workflow

### Step 1: Authentication with RDP APIs

Refinitiv Data Platform entitlement check is based on OAuth 2.0 specification. The first step of an application workflow is to get a token from RDP Auth Service, which will allow access to the protected resource, i.e. data REST API. 

The API requires the following access credential information:
- Username: The username. 
- Password: Password associated with the username. 
- Client ID: This is also known as ```AppKey```, and it is generated using an App key Generator. This unique identifier is defined for the user or application and is deemed confidential (not shared between users). The client_id parameter can be passed in the request body or as an “Authorization” request header that is encoded as base64.

The HTTP request for the RDP APIs Authentication service is as follows:

``` HTTP
POST /auth/oauth2/v1/token HTTP/1.1
Accept: */*
Content-Type: application/x-www-form-urlencoded
Host: api.refinitiv.com:443
Content-Length: XXX

username=RDP_USERNAME
&password=RDP_PASSWORD
&client_id=RDP_APP_KEY
&grant_type=password
&takeExclusiveSignOnControl=true
&scope=trapi
```

Once the authentication success, the function gets the RDP Auth service response message and keeps the following RDP token information in the variables.
- **access_token**: The token used to invoke REST data API calls as described above. The application must keep this credential for further RDP APIs requests.
- **refresh_token**: Refresh token to be used for obtaining an updated access token before expiration. The application must keep this credential for access token renewal.
- **expires_in**: Access token validity time in seconds.

Next, after the application received the Access Token (and authorization token) from RDP Auth Service, all subsequent REST API calls will use this token to get the data. Please find more detail regarding RDP APIs workflow in the following resources:
- [RDP APIs: Introduction to the Request-Response API](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/tutorials#introduction-to-the-request-response-api) page.
- [RDP APIs: Authorization - All about tokens](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/tutorials#authorization-all-about-tokens) page.

In [3]:
#step 1 - get RDP Access Token from RDP

import http.client
import requests 
import json

# Send HTTP Request
auth_url = f'{RDP_HOST}/auth/oauth2/v1/token'
payload = f'grant_type=password&username={username}&client_id={clientId}&password={password}&takeExclusiveSignOnControl=True&scope=trapi'
try:
    response = requests.post(auth_url, 
                             headers = {'Content-Type':'application/x-www-form-urlencoded'}, 
                             data = payload, 
                             auth = (clientId, '')
                )
except requests.exceptions.RequestException as exp:
    print(f'Caught exception: {exp}')


if response.status_code == 200:  # HTTP Status 'OK'
    print('Authentication success')
    access_token = response.json()['access_token']
    refresh_token = response.json()['refresh_token']
    expires_in = int(response.json()['expires_in'])

if response.status_code != 200:
    print(f'RDP authentication failure: {response.status_code} {response.reason}')
    print(f'Text: {response.text}')

Authentication success


## <a id="rdp_get_data"></a>Requesting Data from RDP APIs

That brings us to requesting the RDP APIs data. All subsequent REST API calls use the Access Token via the *Authorization* HTTP request message header as shown below to get the data. 
- Header: 
    * Authorization = ```Bearer <RDP Access Token>```

Please notice *the space* between the ```Bearer``` and ```RDP Access Token``` values.

The application then creates a request message in a JSON message format or URL query parameter based on the interested service and sends it as an HTTP request message to the Service Endpoint. Developers can get RDP APIs the Service Endpoint, HTTP operations, and parameters from Refinitiv Data Platform's [API Playground page](https://api.refinitiv.com/) - which is an interactive documentation site developers can access once they have a valid Refinitiv Data Platform account.

## <a id="rdp_get_green_bulk"></a>Requesting Bulk Green Revenues Data

### Step 2: Listing the packageId of the Bulk Green Revenues Data

To request the Green Revenues Bulk data, the first step is to send an HTTP ```GET``` request to the RDP ```/file-store/v1/file-sets?bucket=bulk-greenrevenue``` endpoint to list all FileSets.

The HTTP Request structure is as follows:

```HTTP
GET /file-store/v1/file-sets?bucket=bulk-greenrevenue HTTP/1.1
Host: api.refinitiv.com
Authorization: Bearer <Acces Token>
```

In [4]:
#step 2 - list Package IDs from bucket name

CFS_url = f'{RDP_HOST}/file-store/v1/file-sets?bucket=bulk-greenrevenue'

try:
    response = requests.get(CFS_url, headers={'Authorization': f'Bearer {access_token}'})
except requests.exceptions.RequestException as exp:
    print(f'Caught exception: {exp}')


if response.status_code == 200:  # HTTP Status 'OK'
    print('Receive list Package IDs from RDP APIs')
else:
    print(f'RDP APIs: CFS request failure: {response.status_code} {response.reason}')
    print(f'Text: {response.text}')

Receive list Package IDs from RDP APIs


Example of the first entry of package IDs, the pacakgeId is the ```packageId``` field.

In [5]:
print(json.dumps(response.json()['value'][0], sort_keys=True, indent=2, separators=(',', ':')))

{
  "attributes":[
    {
      "name":"ContentType",
      "value":"GR Global Summary Full"
    }
  ],
  "availableFrom":"2023-10-15T21:02:50Z",
  "availableTo":"2023-11-15T21:02:50Z",
  "bucketName":"bulk-GreenRevenue",
  "contentFrom":"2023-10-08T20:55:00Z",
  "contentTo":"2023-10-15T20:55:00Z",
  "created":"2023-10-15T21:02:50Z",
  "files":[
    "4193-ab86-cdad0075-b8bb-863b8f13b68a"
  ],
  "id":"4028-3651-1e48ec4a-a823-7a6abe643b2a",
  "modified":"2023-10-15T21:02:53Z",
  "name":"Bulk-GR-Global-Summary-Full-v1-Jsonl-Delta-2023-10-15T21:01:28.093Z",
  "numFiles":1,
  "packageId":"4e94-6d63-fea034dc-90e2-de33895bd4e9",
  "status":"READY"
}


The next step is choosing the package Id. Based on the [Green Revenues User Guide](https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#green-revenues-user-guide), the package Ids are the following:
- Package Name: Bulk-GR-Global-Summary-Full-v1: package Id *4e94-6d63-fea034dc-90e2-de33895bd4e9*
- Package Name: Bulk-GR-Global-Standard-Full-v1: package Id *4316-d43b-81c40763-8e6a-0dbec8162ab1*

In [6]:
packageId = response.json()['value'][0]['packageId']
packageId

'4e94-6d63-fea034dc-90e2-de33895bd4e9'

### Step 3: Listing the Filesets of the Bulk Green Revenues Data with the packageId

The next step is calling the CFS API with the buket name and package Id to list all FileSets using **the package Id**.

API endpint is ```/file-store/v1/file-sets?bucket=bulk-greenrevenue&packageId={packageId}```

The HTTP Request structure is as follows:

``` HTTP
GET /file-store/v1/file-sets?bucket=bulk-greenrevenue&packageId={packageId} HTTP/1.1
Host: api.refinitiv.com
Authorization: Bearer <Access Token>
```

In [7]:
#step 3 - get file id from bucket name

CFS_url = f'{RDP_HOST}/file-store/v1/file-sets?bucket=bulk-greenrevenue&packageId={packageId}'

try:
    response = requests.get(CFS_url, headers={'Authorization': f'Bearer {access_token}'})
except requests.exceptions.RequestException as exp:
    print(f'Caught exception: {exp}')


if response.status_code == 200:  # HTTP Status 'OK'
    print('Receive FileSets list from RDP APIs')
else:
    print(f'RDP APIs: CFS request failure: {response.status_code} {response.reason}')
    print(f'Text: {response.text}')

Receive FileSets list from RDP APIs


In [8]:
print(json.dumps(response.json()['value'][0], sort_keys=True, indent=2, separators=(',', ':')))

{
  "attributes":[
    {
      "name":"ContentType",
      "value":"GR Global Summary Full"
    }
  ],
  "availableFrom":"2023-10-15T21:02:50Z",
  "availableTo":"2023-11-15T21:02:50Z",
  "bucketName":"bulk-GreenRevenue",
  "contentFrom":"2023-10-08T20:55:00Z",
  "contentTo":"2023-10-15T20:55:00Z",
  "created":"2023-10-15T21:02:50Z",
  "files":[
    "4193-ab86-cdad0075-b8bb-863b8f13b68a"
  ],
  "id":"4028-3651-1e48ec4a-a823-7a6abe643b2a",
  "modified":"2023-10-15T21:02:53Z",
  "name":"Bulk-GR-Global-Summary-Full-v1-Jsonl-Delta-2023-10-15T21:01:28.093Z",
  "numFiles":1,
  "packageId":"4e94-6d63-fea034dc-90e2-de33895bd4e9",
  "status":"READY"
}


The File ID is in the ```files``` array

In [9]:
file_id = response.json()['value'][0]['files'][0]
file_id

'4193-ab86-cdad0075-b8bb-863b8f13b68a'

### Step 4: Get the Bulk file URL on AWS S3

The last step is downloading the FIle using File ID with the RDP ```/file-store/v1/files/{file ID}/stream``` endpoint.

The HTTP Request structure is as follows:

``` HTTP
GET /file-store/v1/files/{fileId}/stream?doNotRedirect=true HTTP/1.1
Host: api.refinitiv.com
Authorization: Bearer <Access Token>
```

In [10]:
#step 3 - get file stream (content) from file id

FileID_url = f'{RDP_HOST}/file-store/v1/files/{file_id}/stream?doNotRedirect=true'

try:
    response = requests.get(FileID_url, headers={'Authorization': f'Bearer {access_token}'})
except requests.exceptions.RequestException as exp:
    print(f'Caught exception: {exp}')


if response.status_code == 200:  # HTTP Status 'OK'
    print('Receive File URL from RDP APIs')
else:
    print(f'RDP APIs: CFS request failure: {response.status_code} {response.reason}')
    print(f'Text: {response.text}')

Receive File URL from RDP APIs


The File URL is in the ```url``` attribute.

In [11]:
file_url = response.json()['url']
file_url

'https://a206464-bulk-greenrevenue.s3.amazonaws.com/Bulk-GR-Global-Summary-Full-v1/2023/10/15/Bulk-GR-Global-Summary-Full-v1-Delta-2023-10-15T21%3A01%3A28.093Z.jsonl.gz?x-request-Id=35140d3d-9f39-4d98-ba5c-0c49d24cee7d&x-package-id=4e94-6d63-fea034dc-90e2-de33895bd4e9&x-client-app-id=b4842f3904fb4a1fa18234796368799086c63541&x-file-name=Bulk-GR-Global-Summary-Full-v1-Delta-2023-10-15T21%3A01%3A28.093Z.jsonl.gz&x-fileset-id=4028-3651-1e48ec4a-a823-7a6abe643b2a&x-bucket-name=bulk-GreenRevenue&x-uuid=GESG1-103676&x-file-Id=4193-ab86-cdad0075-b8bb-863b8f13b68a&x-fileset-name=Bulk-GR-Global-Summary-Full-v1-Jsonl-Delta-2023-10-15T21%3A01%3A28.093Z&x-event-external-name=cfs-claimCheck-download&X-Amz-Security-Token=IQoJb3JpZ2luX2VjEKT%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaCXVzLWVhc3QtMSJIMEYCIQD%2F%2BCyOrSqx6LhFmZH8FtvttOXSbnj8JwoVpjQlkVfLTwIhANV6lASaVkIN0%2FN1K3i2074DXUnlvC6iXRyN91mJHdetKqMCCM3%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEQBBoMNjQyMTU3MTgxMzI2Igy1sAJMFyM6ya2HReUq9wHKiitI75mmB%2BGzJTIPFG7708dCICmL

### Step 5: Downloading the file

Based on the S3 ```file_url``` above, the actual file name is *Bulk-GR-Global-Summary-Full-v1-Delta-2023-10-15T21_01_28.093Z.jsonl.gz**. So you need to replace the escape character ```%3A``` with ```_``` (underscore) character.

In [13]:
#Downlaod file
zipfilename = file_url.split("?")[0].split("/")[-1].replace("%3A","_")
print(f'Downloading File {zipfilename} ...')

try:
    response = requests.get(file_url)
except requests.exceptions.RequestException as exp:
    print(f'Caught exception: {exp}')

if response.status_code == 200:  # HTTP Status 'OK'
    print('Receive File Successfully')
    open(zipfilename, 'wb').write(response.content)
    print(f'{zipfilename} Saved')
else:
    print(f'RDP APIs: Request file failure: {response.status_code} {response.reason}')
    print(f'Text: {response.text}')

Receive File Successfully
Bulk-GR-Global-Summary-Full-v1-Delta-2023-10-15T21_01_28.093Z.jsonl.gz Saved


And then unzip the file.

In [16]:
#unzip file
import gzip
import shutil
try:
    unzipfilename = zipfilename.split('.gz')[0]
    print(f'Unzip to {unzipfilename} ...')
    with gzip.open(zipfilename, 'rb') as f_in:
        with open(unzipfilename, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
    print('Done')
except Exception as e:
    print('The error is: ',e)

Unzip to Bulk-GR-Global-Summary-Full-v1-Delta-2023-10-15T21_01_28.093Z.jsonl ...
Done


In [15]:
# View some data
with open(unzipfilename) as f:
    data = f.read(2000)
    print(data)

{"ObjectId":"4295883716;253","GreenRevenueFinancialPeriod":{"ObjectID":"4295883716;253","OrganizationId":"4295883716","PeriodEndDate":"2021-12-31","PeriodFiscalYear":2021,"PeriodId":"253","PeriodStartDate":"2021-01-01"},"GreenRevenue":{"CreatedDatetime":"2023-09-21T00:00:00.000Z","EUTaxonomyEligibilityRevenuePercent":"0","EstimationTypeCode":"Disclosed","GreenRevenuePercent":"0","LastModifiedDatetime":"2023-10-09T03:52:32.517Z","MaximumGreenRevenueExNuclearPercent":null,"MaximumGreenRevenuePercent":"0","MinimumGreenRevenueExNuclearPercent":"0","MinimumGreenRevenuePercent":"0","ReportedRevenueAmount":"606038000","ReportedRevenueCurrencyId":"500207","GreenRevenueCurrencyIdDescription":"MYR","Tier1And2GreenRevenuePercent":null,"Tier1GreenRevenuePercent":null,"Tier2GreenRevenuePercent":null,"Tier3GreenRevenuePercent":null}}
{"ObjectId":"5041758706;77","GreenRevenueFinancialPeriod":{"ObjectID":"5041758706;77","OrganizationId":"5041758706","PeriodEndDate":"2018-12-31","PeriodFiscalYear":2018